In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

Add appts, last month/year?, treasury yield, home sales?, and more stocks?

In [2]:
data = pd.read_csv("data/out/features.csv")
len(data), len(data.columns)

(50, 68)

In [3]:
X, y = data.drop(columns = ['n']), data['n']

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state=69)

In [5]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

In [6]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.0, 5, 0.5)},
                      cv = 5,
                      scoring = 'neg_mean_squared_error',
                      verbose = 3)

In [7]:
search.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .............model__alpha=0.0;, score=-23272.194 total time=   0.0s
[CV 2/5] END ..............model__alpha=0.0;, score=-2425.408 total time=   0.0s
[CV 3/5] END ..............model__alpha=0.0;, score=-6654.062 total time=   0.0s
[CV 4/5] END .............model__alpha=0.0;, score=-31208.388 total time=   0.0s
[CV 5/5] END .............model__alpha=0.0;, score=-85371.290 total time=   0.0s
[CV 1/5] END ..............model__alpha=0.5;, score=-2975.766 total time=   0.0s
[CV 2/5] END ..............model__alpha=0.5;, score=-1407.912 total time=   0.0s
[CV 3/5] END ..............model__alpha=0.5;, score=-4746.786 total time=   0.0s
[CV 4/5] END ..............model__alpha=0.5;, score=-4342.837 total time=   0.0s
[CV 5/5] END .............model__alpha=0.5;, score=-13855.763 total time=   0.0s
[CV 1/5] END ..............model__alpha=1.0;, score=-4132.432 total time=   0.0s
[CV 2/5] END ..............model__alpha=1.0;, sc

C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpecte

[CV 5/5] END ..............model__alpha=1.5;, score=-5693.122 total time=   0.0s
[CV 1/5] END ..............model__alpha=2.0;, score=-3530.079 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.0;, score=-1223.411 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.0;, score=-4000.673 total time=   0.0s
[CV 4/5] END ..............model__alpha=2.0;, score=-3863.321 total time=   0.0s
[CV 5/5] END ..............model__alpha=2.0;, score=-5071.082 total time=   0.0s
[CV 1/5] END ..............model__alpha=2.5;, score=-2743.402 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.5;, score=-1133.724 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.5;, score=-3924.863 total time=   0.0s
[CV 4/5] END ..............model__alpha=2.5;, score=-3555.205 total time=   0.0s
[CV 5/5] END ..............model__alpha=2.5;, score=-4393.533 total time=   0.0s
[CV 1/5] END ..............model__alpha=3.0;, score=-2886.242 total time=   0.0s
[CV 2/5] END ..............m

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])},
             scoring='neg_mean_squared_error', verbose=3)

In [8]:
search.best_params_

{'model__alpha': 4.5}

In [9]:
coef = search.best_estimator_[1].coef_

In [10]:
out = X.iloc[:, coef != 0]
out.columns

Index(['oil_lag12', 'ngf2_lag6', 'mfrm_lag6', 'lfrm_lag3', 'durable_lag12',
       'altsales_lag3', 'miles_lag6', 'GM_lag12', 'GM_v_lag6', 'AN_v_lag3',
       'TSLA_lag12', 'TSLA_v_lag6', 'TSLA_v_lag12', 'g_best_new_cars_lag3'],
      dtype='object')

In [11]:
out.insert(0, 'n', y, True)
out.insert(0, 'month', X['month'], True)

In [12]:
out.to_csv("data/out/lasso.csv")

In [13]:
out

,month,n,oil_lag12,ngf2_lag6,mfrm_lag6,lfrm_lag3,durable_lag12,altsales_lag3,miles_lag6,GM_lag12,GM_v_lag6,AN_v_lag3,TSLA_lag12,TSLA_v_lag6,TSLA_v_lag12,g_best_new_cars_lag3
0,1,352,44.83,22,3.17,3.76,224061,17.818,278372,32.62,421897537,19192133,203.60,80060910,90121332,67
1,2,330,56.83,21,3.06,3.95,218966,17.863,272209,37.31,388156191,15208883,203.34,113858937,106717638,68
2,3,323,51.06,62,3.08,4.01,227573,17.081,255090,37.50,356102158,17266632,188.77,80218666,122990032,77
3,4,369,62.21,64,2.98,3.79,224339,17.610,268469,35.06,324318496,39619066,226.05,100854710,98945034,77
4,5,392,60.47,27,3.18,3.62,217332,17.640,248843,35.97,211953154,45371013,250.80,78487987,89127783,73
5,6,329,58.79,103,3.24,3.71,232766,16.826,259424,33.33,228926730,33490119,268.26,59852189,65473162,76
6,7,420,50.51,56,3.07,3.66,230189,17.247,239679,31.51,376807104,40381923,266.15,79247152,80060910,75
7,8,359,47.01,37,2.93,3.64,222647,17.288,223011,29.44,327501916,23858141,249.06,133705697,113858937,72
8,9,368,43.58,57,2.98,3.48,218738,17.334,265147,30.02,269313331,26540639,248.40,102729544,80218666,70
9,10,367,43.66,82,2.89,3.48,226032,17.744,269653,34.91,249296092,25137021,206.93,135507264,100854710,72
